# Convert a .txt.gz file into .csv file

In [8]:
import pandas as pd
import gzip

edges = []

filepath = '../data/facebook_combined.txt.gz'
filename = filepath.split('/')[-1][:-7] + '.csv'

with gzip.open(filepath, 'rt') as file:
    for line in file:
        if line.startswith('#'):
            continue
        node1, node2 = map(int, line.strip().split())
        edges.append((node1, node2))

df = pd.DataFrame(edges, columns=['node1', 'node2'])
df.to_csv('../data/' + filename, index=False)

# Convert a .edges file into a .csv file

In [1]:
import pandas as pd
import re

edges = []

filepath_input = '../data/proteins/PROTEINS-full.edges'
filepath_output = '../data/proteins/PROTEINS-full.csv'

with open(filepath_input, 'rt') as f:
        for line in f:
            if line.startswith('%'):  # Skip comment lines
                continue
            # Split the line based on spaces or commas
            data = re.split(r'[,\s]+', line.strip())
            if len(data) < 2:  # Skip lines that don't have at least two columns
                continue
            # Extract the first two columns (nodes)
            node1, node2 = int(data[0]), int(data[1])
            edges.append((node1, node2))
            
df = pd.DataFrame(edges, columns=['node1', 'node2'])
df.to_csv(filepath_output, index=False)

# Conveert from .csv to .npy

In [4]:
import numpy as np

folder = '../result/'
filename = 'embeddings_facebook_AW_128.csv'

vec = np.loadtxt(folder + filename, delimiter=',', skiprows=1)
np.save(f"{folder + filename[:-4]}.npy", vec)